In [20]:
import joblib
import utils_precificacao
import pandas as pd
import numpy as np


In [21]:
model = joblib.load('numpy_models/1_model_random_forest_simple.pkl')
enc = joblib.load('numpy_models/enc_for_1_districts.pkl')

In [22]:
n_variables = ['area_util', 'quartos', 'banheiros', 'vagas_na_garagem', 'condominio', 'iptu']

cat_variables = ['bairro']
columns = n_variables + cat_variables

In [56]:
df = utils_precificacao.handle_dataframe("assets/olx_location_cep.json")

df[columns].head()

,area_util,quartos,banheiros,vagas_na_garagem,condominio,iptu,bairro
0,40.0,NaN,0.0,0.0,NaN,NaN,Santa Maria
1,NaN,3.0,2.0,1.0,NaN,NaN,Jardins
2,NaN,3.0,2.0,1.0,NaN,NaN,Jardins
3,61.0,2.0,2.0,1.0,NaN,NaN,Jabotiana
4,NaN,3.0,2.0,1.0,NaN,NaN,Jardins


In [57]:


df_test = df[columns]
df_test = df_test.fillna(df_test.median())
df_test[cat_variables] = enc.transform(df[cat_variables])
df_result = utils_precificacao.apply_model(model, df_test, n_variables + cat_variables)

In [58]:
df_result['previsao']

0       147871.0
1       367439.0
2       367439.0
3       199906.0
4       367439.0
          ...   
5074    215008.0
5075    280029.0
5076    127906.0
5077    186360.0
5078    127082.0
Name: previsao, Length: 5079, dtype: float64

In [61]:
df_to_recommender = pd.read_json("assets/olx_location_cep.json", orient="records", convert_dates=False)

# pd.concat([df_to_recommender,df_result['previsao']], axis=1).to_json("assets/olx_location_cep_price.json", orient="records")

In [60]:
# pd.concat([df,df_result['previsao']], axis=1).to_json("assets/olx_location_cep_price.json", orient="records")

df_check = pd.concat([df,df_result['previsao']], axis=1)
df_check['diff'] = df_check['preco'] -  df_check['previsao']
df_check['percentage'] = np.round((df_check['preco'] -  df_check['previsao'])/(df_check['preco'] + 1), 2)

In [55]:
df_check['created_at']

KeyError: 'created_at'